# 오차역전파

## 연쇄 법칙
    - ㅁㄷㄷㄷ

## 5.1 계산 그래프

- 계산 그래프 : 계산 과정을 그래프로 나타낸 것
- '국소적' : 자신과 직접 관계된 작은 범위
- 국소적 계산 : 전체에서 어떤일이 벌어지든 상관없이 자신과 관계된 정보만으로 결과를 출력할 수 있다는 것
- 계산 그래프의 이점
    - 국소적 계산 : 전체가 아무리 복잡해도 각 노드에서 단순한 계산에 집중하여 문제를 단순화 할 수 있다.
    - 중간 계산 결과 모두 보관 
    - 순전파와 역전파를 이용해 각 변수의 미분을 효율적으로 구할 수 있다!
    



## 5.2 연쇄법칙
- 순전파 : 왼쪽 -> 오른쪽으로 전달
- 역전파 : '국소적인 미분' 을 오른쪽 -> 왼쪽으로 전달
- '국소적인 미분' 을 전달하는 원리는 '연쇄법칙'에 따른 것

### 5.2.1 계산 그래프를 통한 역전파
- 국소적인 미분을 상류에서 전달된 값(E) 에 곱해 앞쪽 노드로 전달

### 5.2.2 연쇄법칙
- 합성 함수의 미분은 합성 함수를 구성하는 각 함수의 미분의 곱으로 나타낼 수 있다

### 5.2 3 연쇄 법칙과 계산 그래프
- 역전파가 하는 일은 연쇄법칙의 원리와 같다
![이미지 설명](https://miro.medium.com/v2/resize:fit:1400/1*oLWpRn3a2IW_YYg9hozS4Q.png)


## 5.3 역전파

- 덧셈 노드의 역전파는 입력된 값을 그래도 다음 노드로 보낸다
- 곱셈 노드의 역전파는 순전파 때의 입력 신호들을 '서로 바꾼 값' 을 곱해서 하류로 보낸다

## 5.4 단순한 계층 구현하기

In [1]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
        
    def forward(self,x,y):
        self.x = x
        self.y= y
        out = x*y
        
        return out
    
    # 상류에서 넘어온 미분(dout)을 순전파 때의 값을 서로 바꿔 곱해 흘린다
    def backward(self, dout):
        dx = dout*self.y
        dy = dout*self.x
        
        return dx,dy

In [2]:
apple = 100
apple_num =2
tax =1.1

# 계층들
mul_apple_layer = MulLayer()
mul_tax_layer= MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price,tax)

print(price)

#역전파
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

220.00000000000003
2.2 110.00000000000001 200


In [3]:
class AddLayer:
    def __init__(self):
        pass
    def forward(self,x,y):
        out=x+y
        return out
    def backward(self,dout):
        dx = dout*1
        dy = dout*1
        return dx,dy

In [4]:
apple = 100
apple_num=2
orange = 150
orange_num =3
tax = 1.1

#계층들
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

#순전파
apple_price = mul_apple_layer.forward(apple, apple_num )
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

print(price)


#역전파
dprice =1
dall_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)
dapple, dpple_num = mul_apple_layer.backward(dapple_price)

print(dapple_num, dapple, dorange, dorange_num,dtax)

715.0000000000001
110.00000000000001 2.2 3.3000000000000003 165.0 650


## 5.5 활성화 함수 계층 구현하기

### 5.5.1 ReLU 계층

![](https://mblogthumb-phinf.pstatic.net/MjAyMTExMDJfMjk2/MDAxNjM1ODI3Njc1NzM0.TfqRMGXfnoMPKfxvK_e6CzOI4-At1-cjoenxEejF5Gwg.1JfhYvoF_RyJ3bxi4IamcR76mThRlzH7EIyjjF1NAA4g.PNG.vail131/image.png?type=w800)

In [5]:
class Relu:
    def __init__(self):
        self.mask = None
        #mask 는 T/F 로 구성된 넘파이 배열
    
    def forward(self,x):
        self.mask = (x <=0)
        out = x.copy()
        out[self.mask] =0
        
        return out
    
    def backward(self,dout):
        dout[self.mask] =0
        dx = dout
        return dx

In [6]:
import numpy as np
x = np.array([[1.0,-0.5],[-2.0,3.0]])
print(x)

mask = (x<=0)

print(mask)

[[ 1.  -0.5]
 [-2.   3. ]]
[[False  True]
 [ True False]]


### 5.5.2 Sigmoid 계층

![](https://as1.ftcdn.net/v2/jpg/05/00/99/90/1000_F_500999006_x0FJCbcDPQwaye1JKG3cYVXSHq0plwIu.jpg)
![](https://heung-bae-lee.github.io/image/differentiation_of_sigmoid_function.png)
1. '/'노드  
    - y =1/x 를 미분하면 -y**2 과 같다
    - 상류에서 흘러온 값에 -y**2 을 곱해서 하류로 전달한다
    
2. '+' 노드 : 상류의 값 그래도 흘려보낸다

3. 'exp' 노드
    - y = exp(x) 연산 수행
    - 미분 : exp(-x) 를 곱해 전달
    
4. 'x' 는 순전파의 값을 서로 바꿔 곱한다

- Sigmoid 계층의 역전파는 순전파의 출력(y) 만으로 계산할 수 있다

In [7]:
class Sigmoid:
    def __init__(self):
        self.out = None
        
    def forward(self,x):
        out = 1/(1+np.exp(-x))
        self.out = out
        return out
    
    def backward(self,dout):
        dx = dout * (1.0 - self.out) * self.out
        return dx

## 5.6 Affine/Softmax 계층 구하기 

### 5.6.1 Affine 계층


- 어파인 변환 : 신경망의 순전파 때 수행하는 행렬의 곱
- 어파인 변환을 수행하는 처리 -> Affine 게층 
- 행렬의 곱에서는 대응하는 차원의 원소 수를 일치시켜야 한다
- 행렬 곱 (dot 노드)의 역전파는 행렬의 대응하는 차원의 원소 수가 일치하도록 곱을 조립해 구할 수 있다
- 편향의 역전파는 두 데이터에 대한 미분을 데이터마다 더해서 구한다 np.sum(dy, axis =0) 0번쨰 축의 총합


In [8]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None

    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b

        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)

        return dx

### Softmax-with-Loss 계층

- 소프트 맥스 함수는 입력 값을 정규화하여 출력
- 신경망 추론에서는 Softmax 계층을 사용하지 않는다
- 신경망에서 정규화되지 않은 출력을 점수(score)이라고 한다
- 신경망 추론에서 답을 하나만 내는 경우에는 가장 높은 점수만 알면 되므로 Softmax 계층 필요 X
- 신경망 학습시에는 필요

#### Softmax-with-Loss 계층
- 소프르맥스 계층에 손실함수인 '교차 엔트로피 오차' 포함
- 3 클래스 분류
    - Softmax 계층은 입역을 정규화해 y 를 출력
    - Cross Entropy Error 계층은 Softmax 의 출려(y) 와 정답 레이블(t)를 받고 손실 L을 출력
    - '소프트 맥스 함수'의 손실함수로 '교차 엔트로피 오차'를 사용하니 역전파 (y-t)로 말끔히 떨어진다
    


In [9]:
class SoftmaxWIthLoss:
    def __init__(self):
        self.loss = None #손실
        self.y=None #softmax 의 출력
        self.t = None #정답 레이블 
        
    def forward(self,x,t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self,y, self.t)
        return self.loss
    
    def backward(self,dout=1):
        batch_size= self.t.shape[0]
        dx = (self.y -self.t)/batch_size
        #역전파 때는 전파하는 값을 배치의 수로 나눠 데이터 1개당 오차를 앞 계층에 전파
        return dx

## 5.7 오차역전파법 구현하기

In [10]:
import sys,os
sys.path.append(os.pardir)
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

In [15]:
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size,
        weight_init_std=0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * \
            np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * \
            np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = \
            Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = \
            Affine(self.params['W2'], self.params['b2'])
        self.lastLayer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)

        return x

    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)

    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1:
            t = np.argmax(t, axis=1)

        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy

    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)

        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])

        return grads

    def gradient(self, x, t):
        # 순전파
        self.loss(x, t)

        # 역전파
        dout = 1
        dout = self.lastLayer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db

        return grads



### 5.7.3 오차역전파법으로 구한 기울기 검증하기

- 기울기를 구하는 방법
   - 수치 미분  
   - 해석적 방법 : 오차역전파법 이용 
   
- 수치 미분 특징 : 느리다, 하지만 구현하기 쉽다
- 기울기 확인: 수치 미분의 결과 오차역전파법의 결과를 비교해 검증하는 작업

In [16]:
import sys,os
sys.path.append(os.pardir)
import numpy as p
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

    # 각 가중치의 차이의 절댓값을 구한 후, 그 절댓값들의 평균을 낸다.
for key in grad_numerical.keys():
    diff = np.average(np.abs(grad_backprop[key] - grad_numerical[key]))
    print(key + ":" + str(diff))

W1:2.020995424157243e-13
b1:7.472109991293538e-13
W2:7.511187661971876e-13
b2:1.194599946741093e-10


### 5.7.4 오차역전파법 사용한 학습 구현하기

In [19]:
import sys
import os
import numpy as np
sys.path.append(os.pardir)
from dataset.mnist import load_mnist


# 데이터 읽기
(x_train, t_train), (x_test, t_test) = \
    load_mnist(normalize=True, one_hot_label=True)
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

# 하이퍼 파라메터
iters_num = 10000  # 반복횟수
train_size = x_train.shape[0]
batch_size = 100  # 미니배치 크기
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

# 1에폭당 반복 수
iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    # print(i)
    # 미니배치 획득
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    # 오차역전파법으로 기울기 계산
    grad = network.gradient(x_batch, t_batch)

    # 매개변수 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]

    # 학습 경과 기록
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

    # 1에폭 당 정확도 계산
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train acc, test acc | " + str(train_acc) + ", " + str(test_acc))


train acc, test acc | 0.15591666666666668, 0.1591
train acc, test acc | 0.9055333333333333, 0.9086
train acc, test acc | 0.92265, 0.9234
train acc, test acc | 0.9343666666666667, 0.9332
train acc, test acc | 0.9441333333333334, 0.9429
train acc, test acc | 0.9515833333333333, 0.9497
train acc, test acc | 0.9577333333333333, 0.9544
train acc, test acc | 0.95935, 0.9566
train acc, test acc | 0.96565, 0.9605
train acc, test acc | 0.9682666666666667, 0.962
train acc, test acc | 0.9699833333333333, 0.9632
train acc, test acc | 0.9724833333333334, 0.9653
train acc, test acc | 0.9736666666666667, 0.9657
train acc, test acc | 0.9743166666666667, 0.9678
train acc, test acc | 0.976, 0.9678
train acc, test acc | 0.9777, 0.9687
train acc, test acc | 0.9786333333333334, 0.9698
